In [12]:
using ElectricGrid
using ControlSystemsBase
using PlotlyJS

In [13]:
CM = [0 1
    -1 0]
    
# Source
R = 1.1e-3
L = 70e-6
C = 250e-6

# Cable
C_b = 1e-4/2
L_b = x->1e-4
R_b = 1e-3

# Load
R_l = 100
C_l = 1e-2
L_l = 1e-2;

parameters = Dict()

grid_properties = Dict()
grid_properties["fs"] =  10e3
grid_properties["v_rms"] = 230
grid_properties["phase"] = 1;
parameters["grid"] = grid_properties

source1 = Dict()
source_list = []

# source1["fltr"] = "LC"
source1["fltr"] = "L"
source1["R1"] = R
source1["L1"] = L
# source1["C"] = C

push!(source_list, source1)

parameters["source"] = source_list

cable = Dict()
cable["R"] = R_b
cable["L"] = L_b
cable["C"] = C_b
cable_list = []

push!(cable_list, cable);
parameters["cable"] = cable_list

load1 = Dict()
load_list = []

load1["impedance"] = "RLC"
load1["R"] = R_l;
load1["L"] = L_l;
load1["C"] = C_l;

push!(load_list, load1);
parameters["load"] = load_list;

In [15]:
nc = NodeConstructor(num_sources=1, num_loads=1, parameters=parameters, CM=CM);

In [16]:
A, B, C, D = GetSystem(nc);
@show A

self.parameters = Dict{Any, Any}("source" => Any[Dict{Any, Any}("L1" => 7.0e-5, "mode" => "Synchronverter", "fltr" => "L", "pwr" => 5000.0, "source_type" => "ideal", "std_asy" => NaN, "σ" => 0.0, "i_limit" => 15.737883846284443, "v_rip" => 0.01537, "v_δ_set" => 0.0, "vdc" => 800, "τv" => 0.002, "k" => 0, "R1" => 0.0011, "v_limit" => 1209.222, "τf" => 0.002, "i_rip" => 0.15, "v_pu_set" => 1.0, "control_type" => "classic", "κ" => 0.0, "Δt" => 0.02, "γ" => 0, "X₀" => 0, "p_set" => 0, "q_set" => 0, "pf" => 0.8)], "grid" => Dict{Any, Any}("f_grid" => 50.0, "Δfmax" => 0.5, "fs" => 10000.0, "process_start" => 0.04, "phase" => 1, "ramp_end" => 0.04, "ΔEmax" => 5, "pwr" => 5000.0, "v_rms" => 230), "load" => Any[Dict{Any, Any}("Z" => 0.0012545437087988717 - 0.3541931634009469im, "C" => 0.01, "i_limit" => 826.4992138897273, "L" => 0.01, "R" => 100, "pwr" => 149352.59524550964, "pf" => -0.003541953851758731, "impedance" => "RLC")], "cable" => Any[Dict{Any, Any}("C" => 5.0e-5, "len" => 0.0539511010

5×5 Matrix{Any}:
   -15.7143  -14285.7        0         0           0
 40000.0          0     -40000.0       0           0
     0             #79        #19       #27        0
     0            0         99.7506   -0.997506  -99.7506
     0            0          0       100.0         0

In [17]:
(rows, columns) = size(A)
AA = Matrix{Any}(undef, (rows, columns))
for row in 1:rows
    for column in 1:columns
        h = A[row, column]
        if isa(h, Number)
            AA[row, column] = x -> h
        else
            AA[row, column] = h
        end
    end
end

AAA(x) = (|>).(x, AA)

AAA([0, 0, 0, 0, 0])

5×5 Matrix{Union{Nothing, Real}}:
   -15.7143  -14285.7            0            0           0
 40000.0          0         -40000.0          0           0
     0             nothing     -10.0     -10000.0         0
     0            0             99.7506      -0.997506  -99.7506
     0            0              0          100.0         0

In [6]:
AAA([0,0,1,0,3])

5×5 Matrix{Union{Nothing, Real}}:
   -15.7143  -14285.7            0            0           0
 40000.0          0         -40000.0          0           0
     0             nothing     -10.0     -10000.0         0
     0            0             99.7506      -0.997506  -99.7506
     0            0              0          100.0         0

In [7]:
ts = 1e-5
Ad = exp(A*ts)
Bd = A \ (Ad - C) * B
sys_d = StateSpace(Ad, Bd, C, D, ts);
GetStateIds(nc)

MethodError: MethodError: no method matching exp(::Matrix{Any})
Closest candidates are:
  exp(!Matched::Union{Float16, Float32, Float64}) at special/exp.jl:326
  exp(!Matched::StridedMatrix{var"#s885"} where var"#s885"<:Union{Float32, Float64, ComplexF32, ComplexF64}) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:569
  exp(!Matched::StridedMatrix{var"#s885"} where var"#s885"<:Union{Integer, Complex{<:Integer}}) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:570
  ...

In [8]:
ns = length(A[1,:]) # get num of states
ni = length(B[1,:]) # get num of inputs
t = collect(0:ts:0.1)
x0 = [0.0 for i = 1:ns]
u = [250.0 for i = 1:length(t)]
uu = [u for i = 1:ni ]
uuu = mapreduce(permutedims, vcat, uu);

In [9]:
xout, _, _, _ = lsim(sys_d,uuu,t,x0=x0);

UndefVarError: UndefVarError: sys_d not defined

In [10]:
layout = Layout(xaxis_title="time", yaxis_title="Amplitude")
x_plot(i) = xout[i,:]
a = ["i_LS","u_CK","i_LK","u_CKL","i_LL"]
trace(i) = scatter(x=t, y=x_plot(i), mode="lines",name = "$(a[i])")

plot([trace(1),trace(2),trace(3),trace(4),trace(5)], layout)

UndefVarError: UndefVarError: xout not defined